In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
df_consumos = (
    spark.read.format("delta").load("/Volumes/workspace/default/raw/tb_consumos")
    .select(
        F.col("consumo_id"),
        F.col("hospede_id"),
        F.col("quarto_id"),
        F.col("data_consumo"),
        F.col("tipo_servico"),
        F.col("descricao"),
        F.round(F.col("valor").cast("double"), 2).alias("valor"),  
        F.col("pago"),
    )
    .withColumn("data_consumo", F.to_date(F.col("data_consumo")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_consumos.limit(5))


In [0]:
catalog_path = "/Volumes/workspace/default/trusted/tb_consumos"

if DeltaTable.isDeltaTable(spark, catalog_path):
    delta_table = DeltaTable.forPath(spark, catalog_path)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_consumos.alias("source"),
        "target.consumo_id = source.consumo_id"  # chave única
    ).whenMatchedUpdate(
        set = {
            # Atualiza todas as colunas exceto update_date
            **{col: f"source.{col}" for col in df_consumos.columns if col != "update_date"},
            # Atualiza a update_date com a data atual
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    df_consumos.write.format("delta").save(catalog_path)